In [1]:
# 建图
import re
import networkx as nx
import json

file_repo_encoded = '../src/repo_encoded.txt'
file_repo_cont = '../GHCrawler/repo_contribution.txt'
file_user_encoded = '../src/user_encoded.txt'
file_adjl = '../graph/adjl.adjlist'
file_edgel_weighted = '../graph/edgel_w.edgelist'
file_edgel = '../graph/edgel.edgelist'
G = nx.Graph()
with open(file_repo_cont, 'r', encoding='utf-8') as cont:
    with open(file_repo_encoded, 'r') as repos:
        with open(file_user_encoded, 'r') as users:
            repo_dict = json.loads(repos.read())
            reponum = len(repo_dict)
            user_dict = json.loads(users.read())
            for line in cont:
                blk = line.find('\t')
                full_name = line[:blk].lower()
                line = line[blk + 1:]
                contributers = json.loads(line)
                conrepo = repo_dict[full_name]
                for contributer in contributers:
                    conuser = user_dict[contributer[0]]
                    G.add_edge(conrepo, conuser, weight=contributer[1])
nx.write_adjlist(G, file_adjl)
nx.write_edgelist(G, file_edgel, data=False)
nx.write_weighted_edgelist(G, file_edgel_weighted)

In [11]:
# 调整边的顺序：仓库 - 贡献者
def order(filename, weighted=False):
    with open(filename, 'r') as readfile:
        edges = []
        i = 0
        for line in readfile:
            nodes = line.split()
            if int(nodes[0]) > int(nodes[1]):
                edges.append([nodes[1], nodes[0]])
            else:
                edges.append([nodes[0], nodes[1]])
            if weighted:
                edges[i].append(nodes[2])
            i += 1
    with open(filename, 'w') as writefile:
        for edge in edges:
            writefile.write(edge[0] + ' ' + edge[1])
            if weighted:
                writefile.write(' ' + edge[2])
            writefile.write('\n')
order(file_edgel)
order(file_edgel_weighted, 1)